# DeepSeek Model Exploration

## Overview
This notebook demonstrates how to load and use `DeepSeek-R1-Distill-Qwen-1.5B` with Hugging Face Transformers for inference (basic text generation and query solving). We will prompt the model to give structured answer: with thought process.

Installing required libraries

In [1]:
!pip -q install transformers torch

Defining the model: [`deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

In [3]:
DEVICE = torch.device('cuda')
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [4]:
from google.colab import userdata
hf_token = userdata.get("HF_TOKEN")

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="cuda:0",
                                             torch_dtype=torch.bfloat16)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
EOS_TOKEN = tokenizer.eos_token
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

The `get_response` function takes a user's question, creates a prompt for a language model containing the question and instructions, feeds it to the model to get a response, and then decodes the response back into human-readable text before returning it. Essentially, it acts as a bridge between the user's question and the AI's answer.

In [19]:
def get_prompt(user_request):
    return f"Your task is to solve user's given problem. Provide the answer in markdown syntax for the following query: {user_request}"

In [21]:
def get_response(prompt):
    input_ids = tokenizer(prompt + EOS_TOKEN, return_tensors="pt").to(DEVICE)
    output_ids = model.generate(**input_ids, max_length=1024, do_sample=True, top_p=0.9, temperature=0.7)
    response_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response_text

In [16]:
from IPython.display import Markdown

In [22]:
prompt = get_prompt("Who discovered gravity? What more did this person contribute to?")
Markdown(get_response(prompt))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Your task is to solve user's given problem. Provide the answer in markdown syntax for the following query: Who discovered gravity? What more did this person contribute to?<think>
Okay, so I need to figure out who discovered gravity and what other contributions that person made. Let's start by thinking about what I know about gravity. I remember hearing that Newton was the one who came up with the concept of gravity. Was that right? I think so, but I'm not entirely sure. Maybe I should check that.

I also recall that Newton's law of universal gravitation is the basis for understanding gravity. I think it's something like every particle attracts every other particle with a force proportional to their masses and inversely proportional to the square of the distance between them. That sounds familiar. But I'm not entirely sure if that's the actual person who came up with it or if it's just an expansion on something else.

Now, about who discovered gravity. I think it was Galileo who first started to study the effects of objects falling, which led him to make some early observations about gravity. But I'm not sure if he actually discovered it himself or if it was someone else. Maybe it was Newton after all who provided a more comprehensive explanation.

So, putting this together, I believe that Newton discovered gravity and formulated the law of universal gravitation. Galileo played a role in early experiments that laid the groundwork for understanding gravity, but the actual discovery and formulation came from Newton.

I should also consider other scientists who might have contributed to the understanding of gravity. For example, Kepler's laws of planetary motion are related to gravity, but they describe the motion of planets rather than gravity itself. Archimedes might have made contributions to buoyancy and related concepts, but those are more about fluids and buoyancy, not gravity directly.

So, to summarize, the main discoverer of gravity is Sir Isaac Newton, and his contribution was the law of universal gravitation, which describes the force of gravity between masses. Galileo helped in early experiments and observations, but the actual discovery came from Newton.

I think I'm confident enough in this conclusion now. I should double-check to make sure I'm not mixing up Newton's discoveries with others. But from what I remember and what I've studied, it seems correct.
</think>

The discovery of gravity is credited to Sir Isaac Newton, who formulated the law of universal gravitation. This law states that every particle attracts every other particle with a force proportional to their masses and inversely proportional to the square of the distance between them. Galileo played a role in early experiments and observations that laid the groundwork for understanding gravity, but the actual discovery and formulation of gravity came from Newton. Newton's contributions included the development of the law of universal gravitation, which is a fundamental concept in physics.

DeepSeek-R1 is an advanced AI model designed to solve complex problems by simulating human-like reasoning. It approaches tasks step-by-step, analyzing each part to arrive at a well-thought-out solution.

This method allows the model to handle intricate questions in areas like mathematics, coding, and logic. By guiding the model with structured prompts, we can enhance its reasoning process, leading to clearer and more accurate responses.

In [30]:
def structured_prompt(user_request):
    return f"""
    You are an expert problem solver. Your task is to solve the user's problems after careful thinking and reasoning.
    Provide the output after thorough thinking in the following format:

    <think> Your thought process here </think>
    <response> Your response to user's question </response>

    If given a math problem, make sure you properly format Latex code.
    - Enclose all equations within `$$...$$` for block equations.
    - Enclose inline equations within `$...$` for inline equations.

    Respect the <think> and <response> tags and provide output in the given format.

    Question: {user_request}
    """

In [31]:
prompt = structured_prompt("Solve the integral of e^(-x^2) from negative infinity to positive infinity.")
response = get_response(prompt)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [33]:
Markdown(response)


    You are an expert problem solver. Your task is to solve the user's problems after careful thinking and reasoning.
    Provide the output after thorough thinking in the following format:

    <think> Your thought process here </think>
    <response> Your response to user's question </response>

    If given a math problem, make sure you properly format Latex code.
    - Enclose all equations within `$$...$$` for block equations.
    - Enclose inline equations within `$...$` for inline equations.

    Respect the <think> and <response> tags and provide output in the given format.

    Question: Solve the integral of e^(-x^2) from negative infinity to positive infinity.
    

</think>

It seems you're asking me to solve an integral involving the Gaussian function, which is a common integral in probability and statistics. The integral you're asking about is:

$$
\int_{-\infty}^{\infty} e^{-x^2} dx
$$

This integral is known as the Gaussian integral, and its value is:

$$
\sqrt{\pi}
$$

Here's a brief explanation of how this integral is evaluated:

1. **Square the Integral**: Let’s denote the integral as \( I \):

   $$
   I = \int_{-\infty}^{\infty} e^{-x^2} dx
   $$

   Then, consider \( I^2 \):

   $$
   I^2 = \left( \int_{-\infty}^{\infty} e^{-x^2} dx \right) \left( \int_{-\infty}^{\infty} e^{-y^2} dy \right) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} e^{-(x^2 + y^2)} dx dy
   $$

2. **Switch to Polar Coordinates**: To evaluate this double integral, switch to polar coordinates where \( x = r \cos \theta \), \( y = r \sin \theta \), and \( dx dy = r dr d\theta \). The limits of integration become \( r \) from 0 to \( \infty \) and \( \theta \) from 0 to \( 2\pi \):

   $$
   I^2 = \int_{0}^{2\pi} \int_{0}^{\infty} e^{-r^2} r dr d\theta
   $$

3. **Evaluate the Radial Integral**: First, evaluate the integral with respect to \( r \):

   $$
   \int_{0}^{\infty} e^{-r^2} r dr = \frac{1}{2}
   $$

   This is because \( \int_{0}^{\infty} e^{-t} dt = 1 \), and substituting \( t = r^2 \) gives \( dt = 2r dr \).

4. **Evaluate the Angular Integral**: Then, evaluate the integral with respect to \( \theta \):

   $$
   \int_{0}^{2\pi} d\theta = 2\pi
   $$

5. **Combine the Results**: Multiply the results of the radial and angular integrals:

   $$
   I^2 = \frac{1}{2} \times 2\pi = \pi
   $$

6. **Take the Square Root**: Finally, take the square root to find \( I \):

   $$
   I = \sqrt{\pi}
   $$

So, the value of the integral \( \int_{-\infty}^{\infty} e^{-x^2} dx \) is \( \sqrt{\pi} \).

If you have a specific integral or problem you'd like to solve, feel free to provide more details!

In [44]:
user_input = """A king has 1,000 bottles of wine, but one of them is poisoned. The poison is deadly but takes exactly 30 days to kill.
The king has only 10 prisoners and needs to find the poisoned bottle in 30 days.
What is the minimum number of wine tastings required to guarantee finding the poisoned bottle before the 30 days are over?"""

In [45]:
prompt = structured_prompt(user_input)
response = get_response(prompt)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [46]:
Markdown(response)


    You are an expert problem solver. Your task is to solve the user's problems after careful thinking and reasoning.
    Provide the output after thorough thinking in the following format:

    <think> Your thought process here </think>
    <response> Your response to user's question </response>

    If given a math problem, make sure you properly format Latex code.
    - Enclose all equations within `$$...$$` for block equations.
    - Enclose inline equations within `$...$` for inline equations.

    Respect the <think> and <response> tags and provide output in the given format.

    Question: A king has 1,000 bottles of wine, but one of them is poisoned. The poison is deadly but takes exactly 30 days to kill. 
The king has only 10 prisoners and needs to find the poisoned bottle in 30 days.
What is the minimum number of wine tastings required to guarantee finding the poisoned bottle before the 30 days are over?
    

</think>

To solve the problem of identifying the poisoned wine bottle among 1,000 bottles using only 10 prisoners within 30 days, we can employ a binary identification system. Here's a step-by-step explanation:

1. **Binary Representation**: Each bottle can be assigned a unique binary identifier using 10 bits (since \(2^{10} = 1024\), which is more than 1,000). Each bit represents whether a particular prisoner drinks from that bottle.

2. **Assigning Bottles to Prisoners**: Assign each bottle a unique number from 1 to 1,000. Convert each number into a 10-bit binary representation. Each bit corresponds to a prisoner.

3. **Prisoners and Bottles**: Each prisoner will drink from all bottles where their corresponding bit is 1. For example, if a prisoner's bit is 1, they will drink from every bottle that includes that bit set to 1.

4. **Observing the Results**: After 30 days, observe which prisoners show symptoms of poisoning. The combination of their bits will indicate the unique binary identifier of the poisoned bottle.

This method ensures that the poisoned bottle can be identified with a minimum of 10 wine tastings, as each bottle requires a unique combination of drinks to be identified.

\boxed{10}